Note: I have solved this in a couple ways, just to be sure:
1. Minimise MSE
2. Find the 0 of the derivative of MSE w/r x0

In [1]:
# Dependencies
import math
from scipy.optimize import minimize_scalar, root_scalar

In [ ]:
SQRT2PI = math.sqrt(2*math.pi)

def phi(x):
    return math.exp(-0.5*x*x) / SQRT2PI

def Phi(x):
    return 0.5 * (1.0 + math.erf(x / math.sqrt(2.0)))

def mu_L(x0):
    if x0 <= 0.0:
        return 0.0
    return (phi(0.0) - phi(x0)) / Phi(x0)

def mu_R(x0):
    return phi(x0) / (1.0 - Phi(x0))

def mu_L_prime(x0):
    if x0 <= 0.0:
        return 0.0
    p  = Phi(x0)
    ph = phi(x0)
    ph0 = phi(0.0)
    return (x0*ph*p - ph*(ph0 - ph)) / (p*p)

def mu_R_prime(x0):
    p  = Phi(x0)
    ph = phi(x0)
    return ((-x0*ph)*(1.0 - p) + ph*ph) / ((1.0 - p)**2)

def dMSE_dx0(x0):
    p  = Phi(x0)
    ph = phi(x0)
    a  = mu_L(x0)
    b  = mu_R(x0)
    a_p = mu_L_prime(x0)
    b_p = mu_R_prime(x0)
    return (a*a - b*b)*ph - 2*a*(a_p*p + a*ph) - 2*b*(b_p*(1.0-p) - b*ph)


In [3]:
dMSE_dx0(0)

-0.25397454373696393

In [4]:
dMSE_dx0(0.705)

4.505809937505045e-05

In [ ]:
SQRT2PI = math.sqrt(2*math.pi)

def phi(x):  # standard normal pdf
    return math.exp(-0.5*x*x) / SQRT2PI

def Phi(x):  # standard normal cdf
    return 0.5 * (1.0 + math.erf(x / math.sqrt(2.0)))

def mse_cart(x0):
    assert x0 >= 0.0
    var_y = 1.5 - 1.0/(2.0*math.pi)
    p = Phi(x0)
    ph = phi(x0)
    mu_L = (phi(0.0) - ph) / p if x0 > 0 else 0.0
    mu_R = ph / (1.0 - p)
    reduction = p * (1.0 - p) * (mu_R - mu_L)**2
    return var_y - reduction

1.1816901138162095

1.0871185593796453

In [ ]:
# minimising the MSE w/r x0
res = minimize_scalar(mse_cart, bounds=(0.0, 5.0), method="bounded")
x0_star = res.x

# finding the zero of the MSE derivative w/r to x0
root = root_scalar(dMSE_dx0, bracket=(0.0, 5.0), method="brentq")
x0_root = root.root

x0_star, x0_root

(0.7048772521416415, 0.7048766337484362)

In [8]:
mse_cart(x0_star), mu_L(x0_star), mu_R(x0_star)

(1.0871185566003503, 0.11553619488829561, 1.2942176935893865)